### Graph based Music Recommender. Task 2

#### Task description
Data description (DataFrames in parquet format)
Location - /data/sample264

Fields: trackId, userId, timestamp, artistId

trackId - id of the track
userId - id of the user
artistId - id of the artist
timestamp - timestamp of the moment the user starts listening to a track
Location - /data/meta

Fields: type, Name, Artist, Id

Type could be “track” or “artist”
Name is the title of the track if the type == “track” and the name of the musician or group if the type == “artist”.
Artist states for the creator of the track in case the type == “track” and for the name of the musician or group in case the type == “artist”.
Id - id of the item
Task
Build the edges of the type “user-track”. Take the amount of times the track was listened by the user as the weight of the edge from the user's vertex to the track’s vertex. For each user take top-1000 and normalize them.

Sort the resulting Data Frame in descending order by the column norm_count, take top 40 rows, select only the columns “id1”, “id2”, sort them in ascending order this time first by “id1”, then by “id2” and print the columns “id1”, “id2” of the resulting dataframe.

In [1]:
import os

from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql import Window

execfile(os.path.join(os.environ["SPARK_HOME"], 'python/pyspark/shell.py'))

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.1.1
      /_/

Using Python version 2.7.12 (default, Nov 19 2016 06:48:10)
SparkSession available as 'spark'.


In [2]:
sparkSession = SparkSession.builder.enableHiveSupport().master("local [2]").getOrCreate()

data = sparkSession.read.parquet("/data/sample264")
meta = sparkSession.read.parquet("/data/meta")

In [3]:
window1 = Window.partitionBy('userId').orderBy(col('count').desc())
window2 = Window.partitionBy('userId')

userToTrack = data.groupBy(col("userId"), col("trackId")) \
                  .count() \
                  .withColumn("rank", rank().over(window1)) \
                  .filter(col("rank") <= 1000) \
                  .withColumn('sum_count', sum(col('count')).over(window2)) \
                  .withColumn('norm_count', col('count') / col('sum_count') * 0.5) \
                  .orderBy(desc("norm_count"), asc("userId"), asc("trackId")) \
                  .limit(40) \
                  .select(col("userId"), col("trackId")) \
                  .collect()

In [4]:
for val in userToTrack:
    print "%s %s" % val

66 965774
116 867268
128 852564
131 880170
195 946408
215 860111
235 897176
300 857973
321 915545
328 943482
333 818202
346 864911
356 961308
428 943572
431 902497
445 831381
488 841340
542 815388
617 946395
649 901672
658 937522
662 881433
698 935934
708 952432
746 879259
747 879259
776 946408
784 806468
806 866581
811 948017
837 799685
901 871513
923 879322
934 940714
957 945183
989 878364
999 967768
1006 962774
1049 849484
1057 920458
